In [3]:
import json

# Caminho para o arquivo JSON original
input_file = r'C:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\Demo.Report\report.json'
output_file = r'C:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\Demo.Report\report_modified.json'

# Função recursiva para buscar bookmarks nos children
def find_bookmarks_recursively(bookmarks, reference_bookmarks):
    found_bookmarks = []
    for bookmark in bookmarks:
        if bookmark.get("displayName") in reference_bookmarks:
            found_bookmarks.append(bookmark)
        # Verificar se há children e buscar recursivamente
        children = bookmark.get("children", [])
        if children:
            found_bookmarks.extend(find_bookmarks_recursively(children, reference_bookmarks))
    return found_bookmarks

# Carregar o arquivo JSON original
with open(input_file, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Verificar se a chave "config" existe e é uma string
if "config" in data and isinstance(data["config"], str):
    # Desserializar o valor da chave "config" (JSON dentro de uma string)
    config = json.loads(data["config"])
    
    # Localizar a lista de bookmarks
    bookmarks = config.get("bookmarks", [])

    # Identificar os bookmarks de referência
    reference_bookmarks = ["P303_BR_VG_ON", "P303_BR_VG_OFF"]  # Substitua pelos nomes dos bookmarks de referência
    reference_bookmark_objects = find_bookmarks_recursively(bookmarks, reference_bookmarks)
    if not reference_bookmark_objects:
        print(f"Bookmarks de referência '{reference_bookmarks}' não encontrados.")
        exit()

    # Mapear os visualContainers do menu de filtros na seção de referência
    reference_section_id = "e39bbc79f812a36a208e"  # Substitua pelo ID da seção de referência
    reference_visual_ids = [
        "d3f462ccc92392b31b24",  # IDs dos visualContainers na seção de referência
        # Adicione mais IDs conforme necessário
    ]

    # Verificar se reference_visual_ids está vazio
    if not reference_visual_ids:
        print("A lista reference_visual_ids está vazia. Verifique os IDs fornecidos.")
        exit()

    # Criar novos bookmarks para outras páginas
    new_bookmarks = []  # Lista para armazenar novos bookmarks
    for bookmark in bookmarks:
        children = bookmark.get("children", [])
        for child in children:
            exploration_state = child.get("explorationState", {})
            sections = exploration_state.get("sections", {})

            # Iterar pelas sections no explorationState
            for section_id, section_data in sections.items():
                if section_id == reference_section_id:
                    continue  # Pular a seção de referência

                # Mapear os visualContainers da nova seção
                visual_containers = section_data.get("visualContainers", {})
                new_visual_ids = list(visual_containers.keys())

                # Verificar se new_visual_ids está vazio
                if not new_visual_ids:
                    print(f"A seção {section_id} não contém visualContainers.")
                    continue

                # Criar cópias dos bookmarks de referência
                for reference_bookmark in reference_bookmark_objects:
                    new_bookmark = reference_bookmark.copy()
                    new_bookmark["displayName"] = f"{new_bookmark['displayName']} - {section_id}"  # Ajustar o nome do bookmark

                    # Ajustar os IDs dos visualContainers no explorationState
                    for ref_id, new_id in zip(reference_visual_ids, new_visual_ids):
                        if ref_id in visual_containers:
                            visual_containers[new_id] = visual_containers.pop(ref_id)

                    # Adicionar o novo bookmark à lista de novos bookmarks
                    new_bookmarks.append(new_bookmark)

    # Adicionar os novos bookmarks ao JSON
    bookmarks.extend(new_bookmarks)

    # Serializar novamente o dicionário "config" para uma string JSON
    try:
        data["config"] = json.dumps(config)
    except TypeError as e:
        print(f"Erro ao serializar o JSON: {e}")
        exit()

    # Salvar o arquivo modificado
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(data, file, indent=4, ensure_ascii=False)

    print(f"Arquivo modificado salvo como '{output_file}'")
else:
    print("A chave 'config' não existe ou não é uma string no arquivo JSON.")

Arquivo modificado salvo como 'C:\Users\renat\Downloads\Material Referência_Estudo\Estudo PBIP\BPIP Demo\Demo.Report\report_modified.json'
